[View in Colaboratory](https://colab.research.google.com/github/edgarbanhesse/ia369y-affective-computing/blob/master/T2_Sentiment_Analysis.ipynb)

## T2 - Análise de Sentimentos em Textos

### Objetivo

O objetivo desta tarefa é expor os alunos aos desafios práticos envolvidos na análise de textos e na atribuição de valores de valência ou rótulos de emoções a sentenças.

Não é objetivo desta tarefa avaliar a acurácia de detecção ou a eficiência do modelo implementado mas a análise crítica do projeto e o amadurecimento em relação ao problema.

### Descrição da Tarefa

Esta tarefa deverá ser realizada individualmente ou em dupla.

Deve-se escolher um entre os dois problemas propostos abaixo.


### Problema escolhido #2

 - Este problema utilizará a mesma base de dados utilizada no SemEval 2007 - 4th International Workshop on Semantic Evaluations, Task 14, Affective Tests.
 - A base de treinamento conta com 250 manchetes em inglês de jornais e websites (Google, CNN, etc.)
 - A cada manchete está associado um score de (0 a 100)  para os rótulos "anger", "disgust", "fear", "joy", "sadness", "surprise"
 - Também será fornecida uma base de testes e os rótulos "golden" fornecidos durante a conferência.
 - Todos os dados podem ser acessados pelo link, abaixo. Mas atenção, considerar apenas os dados referentes  a rótulos de emoções (identificados com sufixo *emotions*). A leitura dos arquivos README é essencial para o entendimento da base. http://web.eecs.umich.edu/~mihalcea/affectivetext/
 - A tarefa consiste em definir a abordagem ao problema, o modelo de classificação, as regras de análise e deverá realizar uma implementação prática do algoritmo definido.


## Iniciando a resolução do Problema

#### 1. Download do Dataset a ser utilizado e descompactação do mesmo.

In [0]:
#!wget http://web.eecs.umich.edu/~mihalcea/downloads/AffectiveText.Semeval.2007.tar.gz
#!tar -zxvf AffectiveText.Semeval.2007.tar.gz
!ls
!ls AffectiveText.trial

!pip install torch torchvision

AffectiveText.Semeval.2007.tar.gz  AffectiveText.trial	wget-log
AffectiveText.test		   sample_data
affectivetext_trial.emotions.gold  affectivetext_trial.xml
affectivetext_trial.valence.gold   README
    100% |████████████████████████████████| 519.5MB 20kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59dfc000 @  0x7f1838e221c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
  Using cached https://files.pythonhosted.org/packages/ca/0d/f00b2885711e08bd71242ebe7b96561e6f6d01fdb4b9dcf4d37e2e13c5e1/torchvision-0.2.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 2.0MB 3.0MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


#### 2. Importação de bibliotecas.

In [0]:
import re
from collections import namedtuple
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

%matplotlib inline

np.warnings.filterwarnings('ignore')

# Download de alguns dataset disponibilizados pelo NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('movie_reviews')
nltk.download('sentence_polarity')
nltk.download('sentiwordnet')
nltk.download('stopwords')
nltk.download('words')

from nltk.corpus import movie_reviews
# from nltk.corpus import sentence_polarity
# from nltk.corpus import sentiwordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

Conjunto = namedtuple('Conjunto', ['uid', 'phrase',
                                   'tokens',
                                   'valence', 'anger',
                                   'disgust', 'fear',
                                   'joy', 'sadness', 'surprise'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package sentence_polarity to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package sentence_polarity is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


### 3. Importando o Dataset do SemEval 2007 para uma estrutura.

 - **Phrases**: id, phrase
 - **Emotions**: id, anger, disgust, fear, joy, sadness, surprise 
 - **Valence**: id, valence
 
Valor de retorno da função load_semeval_dataset:
 
 - **Content**: uid, phrase, valence,  anger, disgust, fear, joy, sadness, surprise

In [0]:
def clean_phrase(phrase):
    phrase = phrase.lower()
    # Remove pontuação
    phrase = re.sub(r'[\"\'!@#$%&*\(\)-_=+{}\[\]:;>.<,|\\`´]', '', phrase)
    # Remover stopwords em inglês e Lematização das palavras
    wordnet_lemmatizer = WordNetLemmatizer()
    # stwords = set(stopwords.words('english'))
    stwords = set(ENGLISH_STOP_WORDS)
    phrase = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in phrase.split() if word not in stwords and len(word) > 2])
    # Remove espaços em branco extras
    phrase = re.sub(r'\s{1,}', ' ', phrase)
    return phrase


def tokenize(phrase):
    # Limpar e retorna trigramas da frase
    return clean_phrase(phrase).split()


def load_semeval_dataset(fname='test'):
    content = []
    phrases = []
    emotions = []
    valences = []
    with open(f'AffectiveText.{fname}/affectivetext_{fname}.xml') as fxml:
        parser = BeautifulSoup(fxml.read())
        phrases = list(filter(None, parser.text.split('\n')))
    with open(f'AffectiveText.{fname}/affectivetext_{fname}.emotions.gold') as femotions:
        emons = list(filter(None, femotions.read().split('\n')))
        for emotion in emons:
            emon = [int(e) for e in emotion.split()]
            emotions.append(emon)
    with open(f'AffectiveText.{fname}/affectivetext_{fname}.valence.gold') as fvalences:
        valens = list(filter(None, fvalences.read().split('\n')))
        for valence in valens:
            valen = [int(v) for v in valence.split()]
            valences.append(valen)
    for i, phrase in enumerate(phrases):
        content.append(Conjunto(uid=valences[i][0],
                                phrase=phrase,
                                tokens=tokenize(phrase),
                                valence=valences[i][1:][0],
                                anger=emotions[i][1],
                                disgust=emotions[i][2],
                                fear=emotions[i][3],
                                joy=emotions[i][4],
                                sadness=emotions[i][5], 
                                surprise=emotions[i][6]))
    return content

# Carregar os dados de test e trial
train = load_semeval_dataset('test')
test = load_semeval_dataset('trial')

### 4. TF-IDF

Implementação do método estatístico TF-IDF e comparativo com a versão implementada pela biblioteca scikit-learn.

Na biblioteca da scikit-learn é aplicada dentre outros procedimentos a normalização nos termos resultantes da tf-idf. A normalização utilizada é a distância euclidiana dos valores. É possível verificar uma explicação detalhada de alguns passos da implementação no [site](http://scikit-learn.org/stable/modules/feature_extraction.html) deles.

Nossa implementação abaixo, segue o padrão apresentado na [wikipedia](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

Onde:

$tfidf(t, d, D) = tf(t,d) \times idf(t, D)$

$tf(t, d) = f_{t,d} / \Sigma{f_{t',d}}$

$idf(d, D) = \log \frac{|D|}{d}$

Sendo $t$ o termo (palavra), $d$ o documento e $D$ o conjunto de documentos.

In [0]:
class TfidfImpl:

    def __init__(self, stopwords=None):
        self.stopwords = stopwords
    
    def clean_phrase(self, phrase):
        phrase = phrase.lower()
        # Remove pontuação
        phrase = re.sub(r'[\"\'!@#$%&*\(\)-_=+{}\[\]:;>.<,|\\`´]', '', phrase)
        if self.stopwords == 'english':
            # Remover stopwords em inglês e Lematização das palavras
            wordnet_lemmatizer = WordNetLemmatizer()
            # stwords = set(stopwords.words('english'))
            stwords = set(ENGLISH_STOP_WORDS)
            phrase = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in phrase.split() if word not in stwords and len(word) > 2])
        # Remove espaços em branco extras
        phrase = re.sub(r'\s{1,}', ' ', phrase)
        return phrase


    def tokenize(self, phrase):
        # Limpar e retorna trigramas da frase
        return self.clean_phrase(phrase).split()
    
    def bag_of_words(self, phrases):
        bow = []
        for phrase in phrases:
            bow += phrase
        return sorted(set(bow))


    def compute_tf(self, words):
        tf = {}
        lbow = len(words)
        for word in words:
            tf[word] = tf.get(word, 0) + 1
        for word, count in tf.items():
            tf[word] = count / lbow
        return tf


    def compute_idf(self, phrases, N, bow):
        idfs = {}
        for df in bow:
            idfs[df] = idfs.get(df, 0)
            for words in phrases:
                if df in words:
                    idfs[df] = idfs.get(df, 0) + 1
        for df in idfs.keys():
            idfs[df] = np.log10(N / idfs[df])
        return idfs


    def compute(self, phrases):
        # Checagem... e conversão
        assert len(phrases) > 0
        if type(phrases[0]) is str:
            phrases = [self.tokenize(phrase) for phrase in phrases]
        
        tf_idf = {}
        N = len(phrases)
        bow = self.bag_of_words(phrases)
        idf = self.compute_idf(phrases, N, bow)
        for words in phrases:
            tf = self.compute_tf(words)
            for word, val in tf.items():
                tf_idf[word] = val * idf[word]
        return tf_idf

Resultados das funções criadas acima:

In [0]:
# Convertendo o conjunto montado acima para apenas as frases
# phrases = []
# conjuntos = test + trial
# for conj in conjuntos:
#    words = conj.tokens
#    phrases.append(' '.join(words))

# Testes  
phrases = ['The cat is in the hole', 'The rat is in the hole', 'The dog are looking at you througth the hole']

# Mostrando apenas as primeiras palavras com menor valor
head = 5

print('Nossa implementação: ')
Tfidf = TfidfImpl(stopwords='english')
tf_idf = Tfidf.compute(phrases)
df = pd.DataFrame({'term': list(tf_idf.keys()), 'weight': list(tf_idf.values())})
df = df.sort_values(by='weight', ascending=True)
display(df.head(head))

print()
print('-' * 20)
print('Implementação do Scikit-Learn: TfidfVectorizer')
vectorizer = TfidfVectorizer(stop_words='english', norm=None, smooth_idf=False)
transformed_weights = vectorizer.fit_transform(phrases)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(by='weight', ascending=True)
display(weights_df.head(head))

print()
print('-' * 20)
print('Implementação do Scikit-Learn: CountVectorizer + TfidfTransformer')
cvec = CountVectorizer(stop_words='english')
cvec.fit(phrases)
cvec_counts = cvec.transform(phrases)
transformer = TfidfTransformer(norm=None, smooth_idf=False)
transformed_weights = transformer.fit_transform(cvec_counts)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df = weights_df.sort_values(by='weight', ascending=True)
display(weights_df.head(head))

Nossa implementação: 


,term,weight
1,hole,0.000000
3,dog,0.119280
4,looking,0.119280
5,througth,0.119280
0,cat,0.238561



--------------------
Implementação do Scikit-Learn: TfidfVectorizer


,term,weight
0,cat,0.699537
1,dog,0.699537
3,looking,0.699537
4,rat,0.699537
5,througth,0.699537



--------------------
Implementação do Scikit-Learn: CountVectorizer + TfidfTransformer


,term,weight
0,cat,0.699537
1,dog,0.699537
3,looking,0.699537
4,rat,0.699537
5,througth,0.699537


In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

def retorna_frases(conjunto):
    frases = [conj.phrase for conj in conjunto]
    valencia = [conj.valence for conj in conjunto]
    return frases, valencia

# Convertendo o conjunto montado acima para apenas as frases
p_train_frases, p_train_valencia = retorna_frases(train)
p_test_frases, p_test_valencia = retorna_frases(test)

predict_frases = [
    'Bad news comes first, good news after.',
    'I enjoy too much this movie, i love it.',
    'Hate you, and your brother!',
    'Great day to walk in the park and play with my dog!',
    'Don\'t you worry my friend.',
    'I have bad news for you sir.',
]

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 5))
train_transformed_weights = vectorizer.fit_transform(p_train_frases)
test_transformed_weights = vectorizer.transform(p_test_frases)
validate_transformed_weights = vectorizer.transform(predict_frases)

modelo = MultinomialNB()
modelo.fit(train_transformed_weights, p_train_valencia)
print(modelo.score(train_transformed_weights, p_train_valencia))
print(modelo.predict(validate_transformed_weights))

0.653
[ 38  47 -64  38  38  38]


In [0]:
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

datasets = np.array(p_train_frases + p_test_frases)
values = np.array(p_train_valencia + p_test_valencia)

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 5))
crossval_transformed_weights = vectorizer.fit_transform(datasets)

predicted = cross_val_predict(modelo, crossval_transformed_weights, values, cv=6)
metrics.accuracy_score(values, predicted)

0.0136